<a href="https://colab.research.google.com/github/majid1363/majid1363/blob/main/IMDB_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let’s apply this idea to the IMDB movie-review sentiment-prediction task that
you’re already familiar with.

In [ ]:
from keras.datasets import imdb
from keras import preprocessing

# Number of words to consider as features
max_features = 5000
# Cuts off the text after this number of words (among the max_features most common words)
max_len = 50

# Loads the data as lists of integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# Turns the lists of integers into a 2D integer tensor of shape (samples, max_len)
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=max_len)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=max_len)

#
print(x_train.shape)
print(x_test.shape)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

(25000, 50)
(25000, 50)


The network will learn 8-dimensional embeddings for each of the 10,000 words, turn the input integer sequences (2D integer tensor) into embedded sequences (3D float tensor), flatten the tensor to 2D, and train a single Dense layer on top for classification.

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dropout, Dense
from keras import regularizers

regularizer = regularizers.l2(l2=1e-5)

model = Sequential()

# Specifies the maximum input length to the Embedding layer so you can later
# flatten the embedded inputs. After the Embedding layer, the activations
# have shape (samples, max_len, 8).
model.add(Embedding(max_features, 8, input_length=max_len))

# Flattens the 3D tensor of embeddings into a 2D tensor of shape (samples, max_len * 8)
model.add(Flatten())

# 
model.add(Dropout(0.5))

# Adds the classifier on top
model.add(Dense(1, activation='sigmoid', kernel_regularizer=regularizer))

# compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()

# fit model
history = model.fit(x_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(x_test, y_test))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 8)             40000     
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dropout (Dropout)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 401       
Total params: 40,401
Trainable params: 40,401
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
782/782 [==============================] - 3s 3ms/step - loss: 0.6812 - acc: 0.5635 - val_loss: 0.5255 - val_acc: 0.7762
Epoch 2/50
782/782 [==============================] - 2s 2ms/step - loss: 0.4827 - acc: 0.7855 - val_loss: 0.4159 - va

You get to a validation accuracy of ~80%, which is pretty good considering that you’re only looking at the first 40 words in every review. But note that merely flattening the embedded sequences and training a single Dense layer on top leads to a model that treats each word in the input sequence separately, without considering inter-word relationships and sentence structure (for example, this model would likely treat both “this movie is a bomb” and “this movie is the bomb” as being negative reviews). It’s much better to add recurrent layers or 1D convolutional layers on top of the embedded sequences to learn features that take into account each sequence as a whole. That’s what we’ll focus on in the next few sections.

In [ ]:
# 
weights = model.layers[0].get_weights()[0]
print(weights.shape)

(5000, 8)
